# **Face Mask Monitoring Training Code Starts Here!** 

### **1: Mounting Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive'
!pwd # should say '/content/gdrive/MyDrive'

### **2: Clone YOLOv5 Repository from official GitHub account**

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo

### **3: Install All Required Dependencies**

In [ ]:
!pip install -U -r yolov5/requirements.txt      # install dependencies (ignore errors)

### **4: Go Into The Cloned Folder**

In [ ]:
%cd /content/gdrive/MyDrive/yolov5

### **5: Setting GPU As Our Default Training Device**

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
%cd /content

### **6: Unzipping Our Dataset Into Test, Train and Validation Accordingly**

In [ ]:
!curl -L "https://app.roboflow.com/ds/CWwFUfO9OA?key=g5mpWNuJ14" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

### **7: We Now Need To Edit The "data.yaml" File And Write Absolute Paths For Our Datasets**

We Need to Make The Following 4 Changes:


1.   train = /content/train/images
2.   val = /content/valid/images
3.   nc = 2
4.   names: ['Mask', 'No Mask']

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat data.yaml

### **8: Defining Number of Classes As Given In The Updated "data.yaml" File**

In [ ]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

### **9: We Will Start Our Training By Using Pre-Trained Weights (yolov5s) And Then Further Training It On Our Dataset**

In [ ]:
%cat /content/gdrive/MyDrive/yolov5/models/yolov5s.yaml

### **10: Some Customizations To Edit Variables**

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/gdrive/MyDrive/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### **11: Installing "wandb" To See Real-Time Analysis Of Our Model During The Training**

In [ ]:
!pip install wandb

### **12: Training Our Model**

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/gdrive/MyDrive/yolov5/
!python train.py --img 640 --batch 64 --epochs 15 --data '/content/data.yaml' --cfg ./models/custom_yolov5s.yaml --weights yolov5s.pt --name yolov5s_final  --cache